In [2]:
import os
import numpy as np
import cv2
import scipy.io
import csv
from matplotlib import pyplot as plt
from tifffile import imwrite, imread

import scipy.ndimage as spim

from MiniscopeProcessingPython import crop_and_convert_miniscope, get_all_CropROIs, getVideosInFolder, GetMiniscopeDirs
from MiniscopeProcessingPython import combine_multi_folders, scroll_images, load_miniscope_avis, crop_and_convert_colorAVI

%matplotlib qt

First we extract the directories with miniscope recordings and behav cam recordings.

Setup the parameters so it knows what to look for and return
Folder structures should look like the miniscope default, i.e.


[topdir]/[animal]/[202*_dayfolder]/[sessionfolder]/[MiniLFOV or BehavCam]/

You might need to change the had coded year and camera names based on your recordings


In [11]:

topdir = 'G:/.shortcut-targets-by-id/1mQqRDGRSlffLsw4MnHKbIatTEeGRdoaY/miniscope_v4/Mov_Grid/'
anames = ['A', 'B', 'C', 'D', 'E', 'F', 'G']
# anames = ['B']
exten = ['_baseline_6mov/camkII_mpfc', '_shocks_12mov/camkII_mpfc', '_retrieval_24mov/camkII_mpfc']
animals = []
for aLoop in anames:
    for eLoop in exten:
        animals.append(str(aLoop + eLoop))
animals

    
miniscope_tiff_name = 'msCam.tiff' # output name of concatenated tiff file to check for
behavcam_tiff_name = 'behavCam.tiff'

# select folders that have/dont have tiff_name in them. Use None to ignore
HasTiff = None # False, True, or None

# This will output the folders
MiniFolders, BehavFolders, parentMiniFolders, _ = \
    GetMiniscopeDirs(topdir, animals, TiffDirsOnly=HasTiff, miniscope_tiff_name=miniscope_tiff_name, behavcam_tiff_name=behavcam_tiff_name,
                     scope_name='My_V4_Miniscope', behavcam_name='My_WebCam')

for f in MiniFolders:
    print("'" + str(f) + "',...")


'G:/.shortcut-targets-by-id/1mQqRDGRSlffLsw4MnHKbIatTEeGRdoaY/miniscope_v4/Mov_Grid/A_baseline_6mov/camkII_mpfc/2023_09_12/15_49_27/My_V4_Miniscope',...
'G:/.shortcut-targets-by-id/1mQqRDGRSlffLsw4MnHKbIatTEeGRdoaY/miniscope_v4/Mov_Grid/A_shocks_12mov/camkII_mpfc/2023_09_13/15_16_35/My_V4_Miniscope',...
'G:/.shortcut-targets-by-id/1mQqRDGRSlffLsw4MnHKbIatTEeGRdoaY/miniscope_v4/Mov_Grid/A_retrieval_24mov/camkII_mpfc/2023_09_15/15_14_12/My_V4_Miniscope',...
'G:/.shortcut-targets-by-id/1mQqRDGRSlffLsw4MnHKbIatTEeGRdoaY/miniscope_v4/Mov_Grid/B_baseline_6mov/camkII_mpfc/2023_09_12/16_16_30/My_V4_Miniscope',...
'G:/.shortcut-targets-by-id/1mQqRDGRSlffLsw4MnHKbIatTEeGRdoaY/miniscope_v4/Mov_Grid/B_shocks_12mov/camkII_mpfc/2023_09_13/15_43_47/My_V4_Miniscope',...
'G:/.shortcut-targets-by-id/1mQqRDGRSlffLsw4MnHKbIatTEeGRdoaY/miniscope_v4/Mov_Grid/B_retrieval_24mov/camkII_mpfc/2023_09_15/15_43_03/My_V4_Miniscope',...
'G:/.shortcut-targets-by-id/1mQqRDGRSlffLsw4MnHKbIatTEeGRdoaY/miniscope_v4/Mov_G

## Next we want to take the multiple avi files and combine them to a single tiff file for analysis
### If the data just needs to be combined and cropped - [multiple avis] > 'msCam.tiff'
1. for each directory in 'MiniFolders', it will first concatenate all of the videos into one array
2. each frame is filtered based on 'ksize'
3. the array is cropped to the priovided ROI (see 'crop_shape' parameter)
4. the  array is then downsampled spatially ('s_ds') and temporally ('t_ds')
5. then saved to a tiff file, 'tiff_name'


### If the data contains line noise, first we combine them with no filtering or downsampling 
[not demonstrated in this demo yet]
### Logic will be as follows:
1. combine the avis into a single unfiltered, undownsampled tiff file. 
            - [multiple avis] > 'noisey.tiff'
2. use the periphery of the 'noisey.tiff' to remove the line noise (subtract the median of the rows)
            - 'noisey.tiff'  >  msCam.tiff

In [ ]:
# miniscopeFolders = allMiniFolders
# check out pathlib library for / \ probs
## PARAMETERS
ksize = 3 # recommend a small filter, 3 pixels usually, done before spatial downsample
kern  = ksize # np.ones([ksize,ksize]) # small median filter to remove salt and pepper
s_ds  = 2 # spatial/pixel down sample
t_ds  = 1 # temporal/frame down sample

video_name = '' # expression labelling miniscope avi files, such as 'msCam' (if 'msCam1.avi', 'msCam2.avi',...)
                # or '' (if '0.avi', '1.avi', etc). Starts at the lowest and goes to the highest
# v4 miniscope==''
# v3 miniscope=='msCam'

# use_prev_crop = True
miniscope_tiff_name = 'msCam.tiff'
tiff_name  = miniscope_tiff_name
crop_shape = 'rect' # 'rect' # 'rect' # 'circle', 'rectangle', or 'square'. For no cropping: crop_shape=None
use_mask   = False # if cropping with a circle, can mask out the area outside of the circle
verbose    = True # more or less print details
# if you use circle but don't want the masking, just use square instead

# Get the crop ROIs for each folder, store in a dictionary 'crop_ROIs'
# mask_ROIs is a dict of circular masks if you want to use circle and mask out the periphery
HPC_MiniFolders
if (crop_shape != None) and (crop_shape != 'prev'):
    cropROIs, maskROIs  = get_all_CropROIs(HPC_MiniFolders, avi_names=video_name, shape=crop_shape, verbose=verbose)
    
for subFolder in HPC_MiniFolders:
    if crop_shape != None: 
        crop = cropROIs[subFolder]
    else:
        crop = None
    print('Crop:  ' + str(crop))
    
    if use_mask: mask=maskROIs[subFolder]
    else: mask=None
    # Now do the actual combining and cropping 
    crop_and_convert_miniscope(dataFolder=subFolder, tiff_name=tiff_name, avi_names=video_name,
            cropROI=crop, circleDef=mask, spatialDownSample=s_ds, temporalDownSample=t_ds,
            frame_filter_size=kern, verbose=verbose, multicore=True, dff=False)

In [5]:
# f = "G:/.shortcut-targets-by-id/1mQqRDGRSlffLsw4MnHKbIatTEeGRdoaY/miniscope_v4/Mov_Grid/G_shocks_12mov/camkII_mpfc/2023_09_26/17_00_14/My_V4_Miniscope/msCam.tiff"
# f = "G:/.shortcut-targets-by-id/1mQqRDGRSlffLsw4MnHKbIatTEeGRdoaY/miniscope_v4/Mov_Grid/G_shocks_12mov/camkII_mpfc/2023_09_26/17_00_14/My_V4_Miniscope/msCam_MC.tiff"
# f = f.replace('\\', '/')
f = subFolder + '/' + tiff_name
Y = imread(f)
scroll_images(Y, resize_val=2) # press q to quit

# Crop and combine HPC miniscope dirs

## BEHAV CAM

In [ ]:

# check out pathlib library for / \ probs
## PARAMETERS
ksize = 1 # recommend a small filter, 3 pixels usually, done before spatial downsample
kern  = None # np.ones([ksize,ksize]) # small median filter to remove salt and pepper
s_ds  = 1 # spatial  down sample
t_ds  = 1 # temporal down sample

video_name = '' # expression labelling miniscope avi files, such as 'msCam' (if 'msCam1.avi', 'msCam2.avi',...)
                # or '' (if '0.avi', '1.avi', etc). Starts at the lowest and goes to the highest

# use_prev_crop = True
miniscope_tiff_name = 'behavCam.mp4' # 'test.tiff' will not save, for debugging
tiff_name  = miniscope_tiff_name
crop_shape = 'rect' # 'rect' # 'rect' # 'circle', 'rectangle', or 'square'. For no cropping: crop_shape=None

use_mask   = False # if cropping with a circle, can mask out the area outside of the circle. Doesn't really work
verbose    = True # more or less print details
# if you use circle but don't want the masking, just use square instead

# Get the crop ROIs for each folder, store in a dictionary 'crop_ROIs'
# mask_ROIs is a dict of circular masks if you want to use circle and mask out the periphery
HPC_MiniFolders
if (crop_shape != None) and (crop_shape != 'prev'):
    cropROIs, maskROIs  = get_all_CropROIs(BehavFolders, avi_names=video_name, shape=crop_shape, verbose=verbose)

# Now use each crop and combine the videos per directory
for subFolder in BehavFolders:
    if crop_shape != None: 
        crop = cropROIs[subFolder]
    else:
        crop = None
    print('Crop:  ' + str(crop))
    
    if use_mask: mask=maskROIs[subFolder]
    else: mask=None
    # Now do the actual combining and cropping 
    crop_and_convert_colorAVI(dataFolder=subFolder, aviOUT_name=tiff_name, avi_names=video_name,
            cropROI=crop, circleDef=mask, spatialDownSample=s_ds, temporalDownSample=t_ds,
            frame_filter_size=kern, verbose=verbose, multicore=False)

## ~ Fin ~